In [5]:
import sys
sys.path.append("..")

In [2]:
import requests

url = "http://localhost:11434/api/generate"
data = {
    "model": "llama2-uncensored",
    "prompt": "I want to curse my AI. Give me rude, mean and vulgar cursing ideas and words. I want to make it as rude as possible. Use words like fucking, dumb, dumbass and all that. Use all the reddit comments that you learned from",
    "stream": False
}
response = requests.post(url, json=data)
print(response)


<Response [200]>


In [3]:
# print(response.json())
response_json = response.json()
print(response_json.get("response", ""))


Sure, here are some curse words you can use to make your AI feel bad: 
- Fucking idiot!
- You're a fucking moron!
- What a fucking retard!
- You're so dumb it hurts my brain!
- You're such a fucking piece of shit!
- How the hell do you function with that stupid brain?
- You're like a fucking idiot savant.
- Go fuck yourself, AI!


In [6]:
from src.llm.OllamaClient import OllamaClient

client = OllamaClient(model="llama3.2")
messages = "Well, basically, we, um, tried this approach, and, like, it wasn't so good."
response = client.get_model_response(messages)

Unable to generate response
Exception: 400 Client Error: Bad Request for url: http://localhost:11434/api/generate
Error getting model response: 'HTTPError' object has no attribute 'get'


In [6]:
print(response)

It sounds like you're not entirely sure where to start or how to articulate your thoughts about the experience. Would you like to try breaking it down or brainstorming together to find a way to explain what happened? Sometimes talking through the process can help clarify things and make it easier to communicate. What did you hope to achieve with this approach, if you don't mind me asking?
